In [1]:
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score,roc_curve
from functions import decile_analysis
print('********* Required Libraries Imported *********')

********* Required Libraries Imported *********


In [2]:
#load model developing data
model_data = pd.read_parquet('Leobrix_Release_BBILAP_Combined_Data.parquet.gzip')
print('********* Data loaded ********* ')
print(model_data.shape)
# Make sure no duplicates in the data
model_data =  model_data.drop_duplicates(subset=['unique_reference_number'])
model_data =  model_data.drop_duplicates(subset=['latest_feature_id'])
print(model_data.shape)

********* Data loaded ********* 
(478328, 2831)
(478328, 2831)


In [3]:
#Create New Target (Good in 60-15 but bad in 90-24)
model_data['G15_B24'] = model_data.apply(
   lambda row: 0 if (row['60_in_15mob'] == 0 and row['90_in_24mob'] == 0)
    else ( 1 if (row['60_in_15mob'] == 0 and row['90_in_24mob'] == 1)
    else 2),axis=1)

model_data= model_data[model_data['G15_B24']!=2]
print('********* New Target Prepared *********')

********* New Target Prepared *********


In [4]:
data = model_data.copy()
# take one copy for model development data
#some bank data have inf very rate but by safe replace 
data.replace([np.inf, -np.inf], np.nan, inplace=True)

In [5]:
#Split the time interval for model building and Validation of oot
model_df1 = data[(data['month_year']>='2021-01') & (data['month_year']<='2022-06')]
oot_df1= data[(data['month_year']>='2022-07') & (data['month_year']<='2022-09')]
print('********* Data Splitted *********')
print(model_df1.shape, oot_df1.shape)

********* Data Splitted *********
(34344, 2832) (10802, 2832)


In [6]:
features_list = ['Num_Enq_all_loans_last_9M',
'Num_of_PERSONAL LOAN_opened_last9M',
'num_PERSONAL LOAN_enq_last_36M',
'num_CONSUMER LOAN_enq_last_36M',
'num_CREDIT CARD_enq_last_18M',
'Total_outstanding_Balance_CREDIT CARD',
'highest_sanction_amount_credit_cards',
'Age_of_oldest_open_account',
'num_PL_LE_1_Lac_opened_in_last_12M',
'num_PL_LE_1_Lac_with_no_delinquency',
'Total_Sanction_amt_PERSONAL LOAN_Opened_last_6M',
'Utilization_PERSONAL LOAN_opened_last_9M',
'Utilization_all',
'num_of_loans_le_1lac_non_credit',
                 
# Reconcilated Features
'percentage_ever_delinquent',
'months_since_most_recent_delinquency_installment',
'percentage_utilization_gt_75',
'No of enq in past 90days',
'Num_Enq_non_BANK_last_12M',
'Num_of_installment_loan_opened',
'num_debits_gt_10000_M3',
'num_credit_gt_salary',
'upi_trans_count',
'ratio_balance_after_7_days_max_credit_M3',
'cash_withdrawal_sum',
'num_cash_withdrawal_last_1_months',
'bounced_transactions_count',
'min_balance_last_3_months',
'days_balance_lt_5000_M2',
'days_balance_lt_10000_M3',
'avg_num_credits_last_2_months',
'Max_Credit_Avg_M1_M2_M3',
'ratio_3th_day_balance_to_max_credit_M2',
'ratio_active_loan_vs_salary',
'num_self_trans_last_3_months',
                 
'month_year',
'G15_B24']

In [7]:

model_df = model_df1[features_list]
oot_df = oot_df1[features_list]

# Separate features (X) and target (y) for Model and OOT datasets
X_model = model_df.drop(columns=['month_year', 'G15_B24']) # Adjust 'target' to your actual target column name
y_model = model_df['G15_B24']

X_oot = oot_df.drop(columns=['month_year', 'G15_B24'])
y_oot = oot_df['G15_B24']

#Handling Null Values
for i in X_model.columns:
    X_model[i].fillna(-9999,inplace=True)

for i in X_oot.columns:
    X_oot[i].fillna(-9999,inplace=True)



/tmp/ipykernel_3038637/2552150070.py:13: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  X_model[i].fillna(-9999,inplace=True)
/tmp/ipykernel_3038637/2552150070.py:16: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try usi

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X_model, y_model, test_size=0.40,random_state=42)

In [9]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((20606, 35), (13738, 35), (20606,), (13738,))

In [10]:
#Split Training data into 80% train and 20% eval
X_train_new, X_eval, y_train_new, y_eval = train_test_split(
    X_train, y_train, test_size=0.2, random_state =42
)
print(X_train_new.shape, X_eval.shape, X_test.shape)

(16484, 35) (4122, 35) (13738, 35)


In [11]:
# 2. **Prepare DMatrix for XGBoost**
dtrain = xgb.DMatrix(X_train_new, label=y_train_new)
deval = xgb.DMatrix(X_eval, label=y_eval)
dtest = xgb.DMatrix(X_test, label=y_test)
doot = xgb.DMatrix(X_oot, label=y_oot)


In [12]:
params = {
    'objective': 'binary:logistic',
    'eval_metric': 'auc',
    'learning_rate': 0.01,
     'max_depth': 2,
    'subsample': 0.4,
    'colsample_bytree': 0.4,
    'lambda': 1,
    'alpha': 1.5,
    'gamma': 4.8,
    'random_state': 24
}

evals = [(dtrain, 'train'), (deval, 'eval')]
xgb_model = xgb.train(
    params,
    dtrain,
    num_boost_round= 200,
    early_stopping_rounds= 50,
    evals=evals,
    verbose_eval=10
)

X_train_pred = X_train_new.copy()
X_eval_pred = X_eval.copy()
X_test_pred = X_test.copy()
X_oot_pred = X_oot.copy()


X_train_pred['y_actual'] = y_train_new
X_train_pred['y_pred'] = xgb_model.predict(dtrain)
X_train_pred['LID'] = model_df1.loc[X_train_new.index,'latest_feature_id'].values

X_eval_pred['y_actual'] = y_eval
X_eval_pred['y_pred'] = xgb_model.predict(deval)
X_eval_pred['LID'] = model_df1.loc[X_eval.index,'latest_feature_id'].values

X_test_pred['y_actual'] = y_test
X_test_pred['y_pred'] = xgb_model.predict(dtest)
X_test_pred['LID'] = model_df1.loc[X_test.index,'latest_feature_id'].values

X_oot_pred['y_actual'] = y_oot
X_oot_pred['y_pred'] = xgb_model.predict(doot)
X_oot_pred['LID'] = oot_df1.loc[X_oot.index,'latest_feature_id'].values

[0]	train-auc:0.58013	eval-auc:0.55536
[10]	train-auc:0.62074	eval-auc:0.61951
[20]	train-auc:0.63259	eval-auc:0.62219
[30]	train-auc:0.63058	eval-auc:0.62613
[40]	train-auc:0.63043	eval-auc:0.62855
[50]	train-auc:0.63155	eval-auc:0.63366
[60]	train-auc:0.63292	eval-auc:0.63437
[70]	train-auc:0.63330	eval-auc:0.63519
[80]	train-auc:0.63410	eval-auc:0.63511
[90]	train-auc:0.63438	eval-auc:0.63542
[100]	train-auc:0.63434	eval-auc:0.63582
[110]	train-auc:0.63623	eval-auc:0.63547
[120]	train-auc:0.63798	eval-auc:0.63537
[130]	train-auc:0.63836	eval-auc:0.63420
[140]	train-auc:0.64108	eval-auc:0.63469
[150]	train-auc:0.64364	eval-auc:0.63569
[151]	train-auc:0.64375	eval-auc:0.63545


In [13]:
# 6. **Run Decile Analysis**
train_summary, train_auc, train_gini, train_ks = decile_analysis(X_train_pred, 'y_pred', 'y_actual')
eval_summary, eval_auc, eval_gini, eval_ks = decile_analysis(X_eval_pred, 'y_pred', 'y_actual')

test_summary, test_auc, test_gini, test_ks = decile_analysis(X_test_pred, 'y_pred', 'y_actual')
oot_summary, oot_auc, oot_gini, oot_ks = decile_analysis(X_oot_pred, 'y_pred', 'y_actual')

# 7. **Print Results**
print(f"Train AUC: {train_auc:.2f}, Gini: {train_gini:.2f}, KS: {train_ks:.2f}")
print(f"Eval AUC: {eval_auc:.2f}, Gini: {eval_gini:.2f}, KS: {eval_ks:.2f}")
print(f"Test AUC: {test_auc:.2f}, Gini: {test_gini:.2f}, KS: {test_ks:.2f}")
print(f"OOT AUC: {oot_auc:.2f}, Gini: {oot_gini:.2f}, KS: {oot_ks:.2f}")


Train AUC: 0.64, Gini: 0.29, KS: 0.21
Eval AUC: 0.64, Gini: 0.27, KS: 0.20
Test AUC: 0.63, Gini: 0.26, KS: 0.18
OOT AUC: 0.62, Gini: 0.25, KS: 0.18


In [14]:
train_summary

mean_actual  mean_prediction  max_prediction  min_prediction  count  \
Decile                                                                        
10         0.149788         0.121069        0.188100        0.107220   1649   
9          0.103762         0.101308        0.107215        0.096221   1648   
8          0.063107         0.091512        0.096219        0.086772   1648   
7          0.063069         0.082887        0.086769        0.079415   1649   
6          0.060073         0.077491        0.079415        0.076052   1648   
5          0.061286         0.075120        0.076052        0.074327   1648   
4          0.043056         0.073672        0.074326        0.073066   1649   
3          0.048573         0.072517        0.073066        0.071983   1647   
2          0.036992         0.071449        0.071981        0.070880   1649   
1          0.024864         0.070119        0.070878        0.068627   1649   

        label_good  label_bad  
Decile                         
10            1402        247  
9             1477        171  
8             1544        104  
7             1545        104  
6             1549         99  
5             1547        101  
4             1578         71  
3             1567         80  
2             1588         61  
1             1608         41

In [15]:
test_summary

mean_actual  mean_prediction  max_prediction  min_prediction  count  \
Decile                                                                        
10         0.122999         0.121462        0.193960        0.107777   1374   
9          0.086608         0.101905        0.107764        0.096500   1374   
8          0.071325         0.091737        0.096500        0.087182   1374   
7          0.064822         0.083078        0.087182        0.079605   1373   
6          0.059680         0.077637        0.079604        0.076213   1374   
5          0.056041         0.075243        0.076211        0.074400   1374   
4          0.050983         0.073737        0.074400        0.073118   1373   
3          0.040787         0.072554        0.073116        0.072030   1373   
2          0.038545         0.071500        0.072030        0.070974   1375   
1          0.021106         0.070181        0.070972        0.068682   1374   

        label_good  label_bad  
Decile                         
10            1205        169  
9             1255        119  
8             1276         98  
7             1284         89  
6             1292         82  
5             1297         77  
4             1303         70  
3             1317         56  
2             1322         53  
1             1345         29

In [16]:
oot_summary

mean_actual  mean_prediction  max_prediction  min_prediction  count  \
Decile                                                                        
10         0.151711         0.123754        0.178572        0.109278   1081   
9          0.109259         0.102435        0.109257        0.096822   1080   
8          0.083333         0.091685        0.096818        0.087000   1080   
7          0.075926         0.083269        0.086996        0.080079   1080   
6          0.075000         0.077912        0.080073        0.076332   1080   
5          0.073148         0.075320        0.076327        0.074452   1080   
4          0.064815         0.073810        0.074451        0.073221   1080   
3          0.050926         0.072673        0.073220        0.072145   1080   
2          0.043640         0.071609        0.072145        0.071026   1077   
1          0.033210         0.070285        0.071025        0.068482   1084   

        label_good  label_bad  
Decile                         
10             917        164  
9              962        118  
8              990         90  
7              998         82  
6              999         81  
5             1001         79  
4             1010         70  
3             1025         55  
2             1030         47  
1             1048         36

In [17]:
feature_importance = xgb_model.get_score(importance_type='weight')
feature_importance_df = pd.DataFrame(
    list(feature_importance.items()),
    columns =['Feature','Importance']
).sort_values(by='Importance',ascending=False)
print(feature_importance_df)

                                             Feature  Importance
0                          Num_Enq_all_loans_last_9M        49.0
18                         Num_Enq_non_BANK_last_12M        48.0
26                        bounced_transactions_count        30.0
1                 Num_of_PERSONAL LOAN_opened_last9M        21.0
22                                   upi_trans_count        21.0
12                                   Utilization_all        19.0
7                         Age_of_oldest_open_account        18.0
28                           days_balance_lt_5000_M2        16.0
30                     avg_num_credits_last_2_months        16.0
17                          No of enq in past 90days        15.0
3                     num_CONSUMER LOAN_enq_last_36M        15.0
21                              num_credit_gt_salary        14.0
9                num_PL_LE_1_Lac_with_no_delinquency        12.0
8                 num_PL_LE_1_Lac_opened_in_last_12M        12.0
11          Utilization_P

In [18]:
feature_importance_df

Feature  Importance
0                          Num_Enq_all_loans_last_9M        49.0
18                         Num_Enq_non_BANK_last_12M        48.0
26                        bounced_transactions_count        30.0
1                 Num_of_PERSONAL LOAN_opened_last9M        21.0
22                                   upi_trans_count        21.0
12                                   Utilization_all        19.0
7                         Age_of_oldest_open_account        18.0
28                           days_balance_lt_5000_M2        16.0
30                     avg_num_credits_last_2_months        16.0
17                          No of enq in past 90days        15.0
3                     num_CONSUMER LOAN_enq_last_36M        15.0
21                              num_credit_gt_salary        14.0
9                num_PL_LE_1_Lac_with_no_delinquency        12.0
8                 num_PL_LE_1_Lac_opened_in_last_12M        12.0
11          Utilization_PERSONAL LOAN_opened_last_9M        11.0
5              Total_outstanding_Balance_CREDIT CARD        10.0
2                     num_PERSONAL LOAN_enq_last_36M         9.0
10   Total_Sanction_amt_PERSONAL LOAN_Opened_last_6M         9.0
29                          days_balance_lt_10000_M3         8.0
24                               cash_withdrawal_sum         7.0
16                      percentage_utilization_gt_75         7.0
13                   num_of_loans_le_1lac_non_credit         5.0
20                            num_debits_gt_10000_M3         4.0
4                       num_CREDIT CARD_enq_last_18M         4.0
6               highest_sanction_amount_credit_cards         3.0
31                           Max_Credit_Avg_M1_M2_M3         3.0
14                        percentage_ever_delinquent         3.0
23          ratio_balance_after_7_days_max_credit_M3         3.0
19                    Num_of_installment_loan_opened         3.0
15  months_since_most_recent_delinquency_installment         2.0
25                 num_cash_withdrawal_last_1_months         2.0
27                         min_balance_last_3_months         2.0
32            ratio_3th_day_balance_to_max_credit_M2         1.0

In [19]:
#Save the Pred
Train = X_train_pred[['LID','y_actual','y_pred']]
Eval = X_eval_pred[['LID','y_actual','y_pred']]
Test = X_test_pred[['LID','y_actual','y_pred']]
Oot = X_oot_pred[['LID','y_actual','y_pred']]
Train.to_csv('LeoPayu_BB_Train_Pred_1524.csv',index=False)
Eval.to_csv('LeoPayu_BB_Eval_Pred_1524.csv',index=False)
Test.to_csv('LeoPayu_BB_Test_Pred_1524.csv',index=False)
Oot.to_csv('LeoPayu_BB_Oot_Pred_1524.csv',index=False)


In [20]:
#Save the Model
xgb_model.save_model('LeoPayu_BB_15_in_24mob.json')
print('Model Saved')

Model Saved
